# Machine Learning: Validação de Modelos

* **Base de Dados:** Características físicas de alguns carros que estão ou estiveram à venda, seus preços de venda, e se foram vendidos ou não. Site de vendas fictício.

* **Sobre o projeto:** Este projeto está sendo desenvolvido com referência no curso [Machine Learning: validação de modelos](https://cursos.alura.com.br/course/machine-learning-validando-modelos) da Alura, que é a continuação direta de um outro curso [Machine Learning: classificação com SKLearn](https://cursos.alura.com.br/course/machine-learning-introducao-a-classificacao-com-sklearn). Por isso, a inicialização deste projeto não terá muitas observações.

## Abertura e Tratamentos dos dados

In [ ]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [ ]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   preco            10000 non-null  float64
 1   vendido          10000 non-null  int64  
 2   idade_do_modelo  10000 non-null  int64  
 3   km_por_ano       10000 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 312.6 KB


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print(f"Treinaremos com {len(treino_x)} elementos e testaremos com {len(teste_x)} elementos")

Treinaremos com 7500 elementos e testaremos com 2500 elementos


## Modelo baseline (Dummy Classifier)

* A taxa de acerto do modelo treinado com Dummy Classifier será utilizada como referência para comparações entre outros modelos.

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy='stratified')
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print(f"A acurácia do Dummy Stratified foi de {acuracia:.2f}%")

A acurácia do Dummy Stratified foi de 50.96%


## Modelo 1: Decision Tree

* Profundidade máxima 2.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print (f"A acurácia foi {acuracia:.2f}%")

A acurácia foi 71.92%


## Efeitos da aleatoriedade

* A SEED escolhida nos algoritmos com fatores aleatórios pode mudar muito a taxa de acerto dos modelos, e pode causar decisões equivocadas em decisões de negócio.

In [ ]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print (f"A acurácia foi {acuracia:.2f}%")

A acurácia foi 76.84%


* O modelo pode ser rodado diversas vezes até que uma estimativa da acurácia seja calculada, com uma margem de acerto, não apenas uma porcentagem fixa altamente relacionada ao fato aleatório.

# Validação cruzada (Cross Validation)

* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html

* Variações nos conjuntos de treino e teste dos modelos.

* k-fold e cross validation.

* O k é determinante na confiabilidade da acurácia do modelo. No geral, quanto mais divisões, mais vezes o modelos será avaliado e mais precisa será a acurácia média. Entretanto, quando mais modelos são treinados, mais cara fica a validação.

In [ ]:
from sklearn.model_selection import cross_validate

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5) # 5 divisões
results

{'fit_time': array([0.00919557, 0.00786018, 0.00701451, 0.00722575, 0.00729847]),
 'score_time': array([0.00185323, 0.00336099, 0.00165582, 0.00182056, 0.00182986]),
 'test_score': array([0.756 , 0.7565, 0.7625, 0.7545, 0.7595])}

* Impressão: tempo para calcular resultados, tempo para calcular nota e resultados dos testes.

In [ ]:
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()

print(f'Accuracy - cross validation (5) = [{(media - 2*desvio_padrao)*100 :.2f}%, {(media + 2*desvio_padrao)*100 :.2f}%]')

Accuracy - cross validation (5) = [75.21%, 76.35%]


* Mesmo alterando-se o SEED, o intervalo dificilmente será alterado.

* A quantidade de divisões da Cross-validation ainda pode alterar o intervalo de acurácia. São indicados valores de 5 ou 10, sendo que 5 é o cv padrão do **cross_validate**.

# Aleatoriedade no *cross_validate*

* O parâmetro **cv** aceita um inteiro ou um gerador de validação cruzada.

* O padrão com o inteiro utiliza o (Stratified)KFold.

## K-Fold

In [1]:
def imprime_resultado(results):
  media = results['test_score'].mean()
  desvio_padrao = results['test_score'].std()

  print(f'Accuracy mean = {media*100:.2f}%')
  print(f'Accuracy interval = [{(media - 2*desvio_padrao)*100 :.2f}%, {(media + 2*desvio_padrao)*100 :.2f}%]')

In [ ]:
from sklearn.model_selection import KFold

SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv)

imprime_resultado(results)

Accuracy media = 75.78%
Accuracy interval = [74.37%, 77.19%]


In [ ]:
SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv)

imprime_resultado(results)

Accuracy media = 75.78%
Accuracy interval = [73.58%, 77.98%]


* A média permaneceu a mesma, mas o intervalo se alterou.

* Há várias alternativas para o KFold (Splitter Classes).

* A divisão dos conjuntos de treino e teste de forma totalmente aleatória pode apresentar um desbalanceamento de classes.
  * Neste dataset, por exemplo, seriam muitos carros não vendidos considerados no treinamento, mas muitos carros que foram vendidos no conjunto de teste. As características dos anúncios que conseguiram vender seus carros não seriam bem aprendidas pelo modelo e os resultados seriam prejudicados.

* O parâmetro **stratified** do **train_test_split** garante balanceamento das classes.

* O KFold não garante o balanceamento.

### Simulação de má distribuição dos dados

In [ ]:
dados_azar = dados.sort_values('vendido', ascending = True)
x_azar = dados_azar[['preco', 'idade_do_modelo', 'km_por_ano']]
y_azar = dados_azar['vendido']

SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv)

imprime_resultado(results)

Accuracy media = 57.84%
Accuracy interval = [34.29%, 81.39%]


* A acurácia do modelo foi péssima, próxima à classificação binária aleatória (50%).

In [ ]:
dados_azar = dados.sort_values('vendido', ascending = True)
x_azar = dados_azar[['preco', 'idade_do_modelo', 'km_por_ano']]
y_azar = dados_azar['vendido']

SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv)

imprime_resultado(results)

Accuracy media = 75.76%
Accuracy interval = [73.02%, 78.50%]


* Nesta situação, apenas a aleatorização (shuffle) do conjunto foi suficiente para gerar resultados muito melhores.

## Stratified K-Fold

* Faz a separação proporcional das classes, sem necessidade do shuffle.

In [ ]:
from sklearn.model_selection import StratifiedKFold

SEED = 158020
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv)

imprime_resultado(results)

Accuracy media = 75.78%
Accuracy interval = [73.83%, 77.73%]


## Sobre Splitter Classes

* Cada Splitter Class tem uma situação em que é mais recomendada.

* https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection

# Gerando dados aleatórios para testes de modelos

## Geração da coluna 'modelo' no dataset

In [ ]:
dados['idade_do_modelo'].unique()

array([18, 20, 12,  3,  4, 11, 16, 10, 19, 15,  2, 17,  9, 13,  7,  6,  5,
       14,  8,  1])

In [ ]:
np.random.seed(SEED)
dados['modelo'] = dados['idade_do_modelo'] + np.random.randint(-2, 3, size = 10000)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,17
1,40557.96,1,20,12622.05362,20
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,2
4,117384.68,1,4,12770.11290,2


In [ ]:
dados['modelo'].unique()

array([17, 20, 12,  2, 10,  9, 18,  4, 16,  6,  5, 21, 15, 13, 11, 14,  8,
        3, 19,  7, 22,  0,  1, -1])

In [ ]:
# lidando com o modelo negativo e nulo
dados['modelo'] = dados['modelo'] + abs(dados['modelo'].min()) + 1
dados['modelo'].unique()

array([19, 22, 14,  4, 12, 11, 20,  6, 18,  8,  7, 23, 17, 15, 13, 16, 10,
        5, 21,  9, 24,  2,  3,  1])

# Validação cruzada usando grupos, simulando a chegada de um modelo novo

## Group K-Fold

* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html

* A coluna que contém os grupos deve ser especificada no parâmetro **groups** da **cross_validate**.

In [ ]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo)
imprime_resultado(results)

Accuracy media = 75.76%
Accuracy interval = [72.91%, 78.60%]


# Importância do Pipeline no Cross-validate

* A Árvore de Decisão não tem necessidade de balanceamento de features, mas alguns outros algoritmos, como o SVM, são muito sensíveis à escala dos valores da features e precisam de normalização.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_normalizado = scaler.transform(treino_x)
teste_x_normalizado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_normalizado, treino_y)
previsoes = modelo.predict(teste_x_normalizado)

acuracia = accuracy_score(teste_y, previsoes)
print(f'A acurácia foi de {acuracia*100:.2f}%')

A acurácia foi de 77.48%


* Para usar a cross-validation com o group k-fold, a normalização **deve** ser feita sobre cada grupo separadamente.

In [ ]:
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

# Definição do Pipeline
# O fit e o predict são executados pelo próprio pipeline
scaler = StandardScaler()
modelo = SVC()
pipeline = Pipeline([
    ('transformacao', scaler),
    ('estimador', modelo)
])

cv = GroupKFold(n_splits = 10)
# é passada toda a pipeline ao invés do modelos para que todos os passos definidos sejam executados em cada um dos groups especificados
results = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados['modelo'])
imprime_resultado(results)

Accuracy media = 76.66%
Accuracy interval = [73.65%, 79.66%]


* A execução é mais demorada, mas a estimativa é mais realista.

# Para obter o modelo final, basta treiná-lo em todos os dados usados na cross-validation (do dataset).